<a href="https://colab.research.google.com/github/ashikita/research_data_title/blob/main/get_research_data_title_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DataCite APIで研究データのタイトルを取得

In [2]:
import requests
from urllib.parse import urlencode

# -----------------------------
# 設定パラメータ（必要に応じて変更）
# -----------------------------
affiliation_id = None             # 所属条件を外す場合は None
resource_type_id = "dataset"      # データセットのみ
has_citations = "1"               # 引用あり
include_affiliation = "true"      # 所属情報を含める（レスポンス整形用）
published_year = "2025"           # 出版年条件

# DataCite推奨: 自身を識別できるUser-Agent/メールを付ける（レート制限優遇）
# 例: "YourScriptName/1.0 (mailto:you@example.org)"
USER_AGENT = "AshikitaCounter/1.0 (mailto:ashikita.takuya.221@m.kyushu-u.ac.jp)"

base = "https://api.datacite.org/dois"

params = {
    "page[size]": 1,  # 1件だけ返してもらえば meta.total が読める
    "resource-type-id": resource_type_id,
    "has-citations": has_citations,
    "affiliation": include_affiliation,  # 表示に影響する拡張（件数には無関係）
    "published": published_year,
}

if affiliation_id:
    params["affiliation-id"] = affiliation_id

url = f"{base}?{urlencode(params)}"

resp = requests.get(
    url,
    headers={
        "Accept": "application/vnd.api+json; version=2",
        "User-Agent": USER_AGENT,
    },
    timeout=30,
)
resp.raise_for_status()
payload = resp.json()

total = payload.get("meta", {}).get("total")
page_size = payload.get("meta", {}).get("page") or 1

print(f"総件数（サーバ側フィルタ適用後）: {total:,} 件")
print(f"問い合わせURL: {url}")

総件数（サーバ側フィルタ適用後）: 179,242 件
問い合わせURL: https://api.datacite.org/dois?page%5Bsize%5D=1&resource-type-id=dataset&has-citations=1&affiliation=true&published=2025


In [3]:
import requests
from urllib.parse import urlencode, quote_plus

# ==========================
# 設定（必要に応じて変更）
# ==========================
affiliation_id = None               # 例: "https://ror.org/05dhe8b71"（不要なら None）
resource_type_id = "dataset"
has_citations = "1"
published_year = "2025"

# 除外したい出版者名（名前表記に依存する点に注意）
exclude_publishers = [
    "HEPData",
    "Cambridge Crystallographic Data Centre",
]

# DataCite推奨：識別可能なUser-Agent（メール付き）でアクセス
USER_AGENT = "AshikitaComparator/1.0 (mailto:you@example.org)"

BASE = "https://api.datacite.org/dois"

def build_params(include_query=False):
    """除外なし／ありの共通パラメータを構築"""
    params = {
        "page[size]": 1,                       # 1件だけ返し、meta.totalを読む
        "resource-type-id": resource_type_id,
        "has-citations": has_citations,
        "published": published_year,
    }
    if affiliation_id:
        params["affiliation-id"] = affiliation_id

    if include_query and exclude_publishers:
        # 例: NOT publisher.name:"HEPData" AND NOT publisher.name:"Cambridge Crystallographic Data Centre"
        not_terms = [f'NOT publisher.name:"{p}"' for p in exclude_publishers]
        params["query"] = " AND ".join(not_terms)

    return params

def fetch_total(params):
    url = f"{BASE}?{urlencode(params, quote_via=quote_plus)}"
    r = requests.get(
        url,
        headers={
            "Accept": "application/vnd.api+json; version=2",
            "User-Agent": USER_AGENT,
        },
        timeout=30,
    )
    r.raise_for_status()
    data = r.json()
    total = data.get("meta", {}).get("total")
    return total, url

# ---- 1) 除外なし ----
params_no_excl = build_params(include_query=False)
total_no_excl, url_no_excl = fetch_total(params_no_excl)

# ---- 2) 除外あり（NOT publisher.name）----
params_with_excl = build_params(include_query=True)
total_with_excl, url_with_excl = fetch_total(params_with_excl)

# ---- 出力 ----
print("=== 条件 ===")
print(f"resource-type-id={resource_type_id}, has-citations={has_citations}, "
      f"published={published_year}, affiliation-id={affiliation_id}")
print(f"除外出版者（クエリでNOT）: {', '.join(exclude_publishers) if exclude_publishers else '(なし)'}\n")

print("=== 問い合わせURL ===")
print(f"除外なし : {url_no_excl}")
print(f"除外あり : {url_with_excl}\n")

print("=== 件数比較（meta.total） ===")
print(f"除外なし : {total_no_excl:,} 件")
print(f"除外あり : {total_with_excl:,} 件")
if isinstance(total_no_excl, int) and isinstance(total_with_excl, int):
    reduced = total_no_excl - total_with_excl
    rate = (reduced / total_no_excl * 100) if total_no_excl else 0
    print(f"差分     : {reduced:,} 件 減（{rate:.2f}%）")

=== 条件 ===
resource-type-id=dataset, has-citations=1, published=2025, affiliation-id=None
除外出版者（クエリでNOT）: HEPData, Cambridge Crystallographic Data Centre

=== 問い合わせURL ===
除外なし : https://api.datacite.org/dois?page%5Bsize%5D=1&resource-type-id=dataset&has-citations=1&published=2025
除外あり : https://api.datacite.org/dois?page%5Bsize%5D=1&resource-type-id=dataset&has-citations=1&published=2025&query=NOT+publisher.name%3A%22HEPData%22+AND+NOT+publisher.name%3A%22Cambridge+Crystallographic+Data+Centre%22

=== 件数比較（meta.total） ===
除外なし : 179,245 件
除外あり : 111,568 件
差分     : 67,677 件 減（37.76%）


In [4]:
import requests
import csv
import time
from urllib.parse import urlencode, quote_plus

# ==========================
# 設定（必要に応じて変更）
# ==========================
resource_type_id = "dataset"
has_citations = "1"
published_year = "2025"
affiliation_id = None                  # 例: "https://ror.org/05dhe8b71"（不要なら None）

# サーバ側で除外したい発行者名（比較は publisher.name の文字列に依存）
exclude_publishers = [
    "HEPData",
    "Cambridge Crystallographic Data Centre",
]

# 出力ファイル
output_file = "datacite_datasets_2025_excluded.tsv"

# レート制限配慮（Identified）
USER_AGENT = "AshikitaHarvester/1.0 (mailto:ashikita.takuya.221@m.kyushu-u.ac.jp)"

# 詳細フィールドを展開するか（レスポンス増大に注意）
INCLUDE_AFFILIATION = False   # True にすると affiliation 構造が配列オブジェクトに（やや重くなる）
INCLUDE_PUBLISHER   = False   # True にすると publisher がオブジェクト化（やや重くなる）

# ==========================
# クエリの構築（NOT publisher.name）
# ==========================
not_terms = [f'NOT publisher.name:"{p}"' for p in exclude_publishers]
query_str = " AND ".join(not_terms) if not_terms else None

params = {
    "page[cursor]": 1,             # カーソル方式で初回
    "page[size]": 1000,            # 上限
    "resource-type-id": resource_type_id,
    "has-citations": has_citations,
    "published": published_year,
}
if affiliation_id:
    params["affiliation-id"] = affiliation_id
if query_str:
    params["query"] = query_str
if INCLUDE_AFFILIATION:
    params["affiliation"] = "true"
if INCLUDE_PUBLISHER:
    params["publisher"] = "true"

BASE = "https://api.datacite.org/dois"
url = f"{BASE}?{urlencode(params, quote_via=quote_plus)}"

session = requests.Session()
headers = {
    "User-Agent": USER_AGENT,
    "Accept": "application/vnd.api+json; version=2",
}

total_downloaded = 0
started = time.time()

with open(output_file, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["doi", "title", "publisher", "publicationYear"])

    while url:
        r = session.get(url, headers=headers, timeout=60)
        if r.status_code == 429:
            # 簡易指数バックオフ＋再試行（最大5回）
            for i in range(1, 6):
                time.sleep(min(16, 2 ** i))
                r = session.get(url, headers=headers, timeout=60)
                if r.status_code != 429:
                    break
        r.raise_for_status()
        payload = r.json()

        for item in payload.get("data", []):
            attr = item.get("attributes", {})
            doi = attr.get("doi", "")
            titles = attr.get("titles") or []
            title = titles[0].get("title", "") if titles else ""
            # publisher は文字列（publisher=true を付けるとオブジェクト）
            publisher = attr.get("publisher", "")
            year = attr.get("publicationYear", "")
            w.writerow([doi, title, publisher, year])
            total_downloaded += 1

        # 次ページ（カーソル）へ
        url = payload.get("links", {}).get("next")

        # レート制限配慮（小刻みな待機）
        time.sleep(0.15)

elapsed = time.time() - started
print(f"✅ 完了: {total_downloaded:,} 件を {output_file} に保存しました。経過 {elapsed:.1f} 秒")

✅ 完了: 111,572 件を datacite_datasets_2025_excluded.tsv に保存しました。経過 839.5 秒


In [ ]:
import requests
import csv
import time

# -----------------------------
# 設定パラメータ（変更可能）
# -----------------------------
affiliation_id = None              # 所属条件を外す場合は None
resource_type_id = "dataset"       # データセットのみ
has_citations = "1"                # 引用あり
include_affiliation = "true"       # 所属情報を含める
page_size = 100                    # 1ページあたりの件数
published_year = "2024"            # 出版年条件
output_file = "output.tsv"         # 出力ファイル名
sleep_interval = 1.0               # リクエスト間隔（秒）
max_records = 5000                 # 出力件数の上限（除外後）
exclude_publishers = [
    "HEPData",
    "Cambridge Crystallographic Data Centre"
]  # 除外する出版者リスト

# ベースURLの構築
base_url = "https://api.datacite.org/dois?"
params = []

if affiliation_id:
    params.append(f"affiliation-id={affiliation_id}")
if resource_type_id:
    params.append(f"resource-type-id={resource_type_id}")
if has_citations:
    params.append(f"has-citations={has_citations}")
if include_affiliation:
    params.append(f"affiliation={include_affiliation}")
if published_year:
    params.append(f"published={published_year}")
params.append(f"page[size]={page_size}")

base_url += "&".join(params)

# 出力ファイルの準備
total_count = 0
start_time = time.time()

with open(output_file, "w", newline='', encoding="utf-8") as file:
    writer = csv.writer(file, delimiter='\t')
    writer.writerow(["doi", "title", "publisher", "publicationYear"])

    url = base_url
    while url and total_count < max_records:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"データ取得失敗: {response.status_code}")
            break

        data = response.json()

        # 各レコードから必要な情報を抽出
        for item in data.get("data", []):
            if total_count >= max_records:
                break
            attr = item.get("attributes", {})
            publisher = attr.get("publisher", "")
            if publisher in exclude_publishers:
                continue  # 除外条件に一致した場合はスキップ

            doi = attr.get("doi", "")
            titles = attr.get("titles", [])
            title = titles[0].get("title", "") if titles else ""
            year = attr.get("publicationYear", "")
            writer.writerow([doi, title, publisher, year])
            total_count += 1

        # 次ページがあるか確認
        url = data.get("links", {}).get("next")

        # レート制限対応：1秒待機
        if url and total_count < max_records:
            time.sleep(sleep_interval)

# 経過時間と件数の表示
elapsed_time = time.time() - start_time
print(f"✅ 完了: {total_count} 件のデータを取得しました（上限 {max_records} 件、除外: {', '.join(exclude_publishers)}）。")
print(f"⏱️ 経過時間: {elapsed_time:.2f} 秒")